# Load necessary packages

In [1]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf

# Organise data

## Import features

In [2]:
! pip install 'git+git://github.com/HR/github-clone#egg=ghclone' &> /dev/null

! ghclone https://github.com/timovijn/ElectricityPriceForecasting/tree/master/LSTM

zsh:1: command not found: ghclone


In [3]:
features = pd.read_pickle(f"./features.pkl")

display(features)

,ID3,VOL,MCP,LOAD,LOAD_F,LOAD_FE,ID3 (-4),ID3 (-5),ID3 (-6),ID3 (-7),...,HOD 14,HOD 15,HOD 16,HOD 17,HOD 18,HOD 19,HOD 20,HOD 21,HOD 22,HOD 23
2015-01-08 01:00:00+00:00,22.953776,439.5,32.32,9008.00,8505.25,502.75,29.934792,61.666667,61.118812,61.370370,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 02:00:00+00:00,23.168355,261.5,31.10,8889.25,8222.25,667.00,29.853669,29.934792,61.666667,61.118812,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 03:00:00+00:00,21.000000,420.5,30.17,8929.25,8122.25,807.00,24.012378,29.853669,29.934792,61.666667,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 04:00:00+00:00,30.000000,460.6,24.54,9423.75,8323.50,1100.25,23.269810,24.012378,29.853669,29.934792,...,0,0,0,0,0,0,0,0,0,0
2015-01-08 05:00:00+00:00,30.000000,250.0,32.00,10884.50,9015.00,1869.50,22.953776,23.269810,24.012378,29.853669,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-30 14:00:00+00:00,53.790740,446.6,46.19,13842.50,15329.25,1486.75,76.370821,87.755884,78.709213,52.958116,...,1,0,0,0,0,0,0,0,0,0
2018-12-30 15:00:00+00:00,59.477646,131.6,47.64,14319.25,15644.50,1325.25,63.690401,76.370821,87.755884,78.709213,...,0,1,0,0,0,0,0,0,0,0
2018-12-30 16:00:00+00:00,59.883829,310.1,55.94,15120.75,16285.75,1165.00,56.170316,63.690401,76.370821,87.755884,...,0,0,1,0,0,0,0,0,0,0
2018-12-30 17:00:00+00:00,59.471501,220.9,58.40,14728.75,15555.75,827.00,51.675229,56.170316,63.690401,76.370821,...,0,0,0,1,0,0,0,0,0,0


## Select features

In [4]:
X = features[['ID3', 'LOAD']]
y = features[['ID3']]

lag_X = range(-72, -3, 1)
lag_y = range(0, 3, 1)

X2 = pd.DataFrame(index = X.index, columns = pd.MultiIndex.from_product([['X'], lag_X, X.columns], names = ['Feature', 'Type', 'Lag']))
X2 = X2.rename_axis('Timestamp')

y2 = pd.DataFrame(index = y.index, columns = pd.MultiIndex.from_product([['y'], lag_y, y.columns], names = ['Feature', 'Type', 'Lag']))
y2 = y2.rename_axis('Timestamp')

frame = pd.merge(y2, X2, left_index = True, right_index = True)

X3 = pd.DataFrame(index = X.index)
y3 = pd.DataFrame(index = y.index)

for c in X.columns:
    for l in lag_X:
        X3[f'{c} ({l})'] = X[f'{c}'].shift(-l)

frame['X'] = X3.values

for c in y.columns:
    for l in lag_y:
        y3[f'{c} ({l})'] = y.shift(-l)

frame['y'] = y3.values

frame = frame.dropna()

display(frame)

Feature                            y                                X  \
Type                              0          1          2         -72   
Lag                              ID3        ID3        ID3        ID3   
Timestamp                                                               
2015-01-11 03:00:00+00:00  24.922597  26.238903  27.002718  22.953776   
2015-01-11 04:00:00+00:00  26.238903  27.002718  30.000000  23.168355   
2015-01-11 05:00:00+00:00  27.002718  30.000000  35.000000  21.000000   
2015-01-11 06:00:00+00:00  30.000000  35.000000  35.000000  30.000000   
2015-01-11 08:00:00+00:00  35.000000  35.000000  35.000000  30.000000   
...                              ...        ...        ...        ...   
2018-12-30 12:00:00+00:00  56.170316  51.675229  53.790740  67.046715   
2018-12-30 13:00:00+00:00  51.675229  53.790740  59.477646  68.057669   
2018-12-30 14:00:00+00:00  53.790740  59.477646  59.883829  69.568080   
2018-12-30 15:00:00+00:00  59.477646  59.883829  59.471501  73.739284   
2018-12-30 16:00:00+00:00  59.883829  59.471501  58.448950  76.254406   

Feature                                                                \
Type                                        -71                   -70   
Lag                             LOAD        ID3       LOAD        ID3   
Timestamp                                                               
2015-01-11 03:00:00+00:00  23.168355  21.000000  30.000000  30.000000   
2015-01-11 04:00:00+00:00  21.000000  30.000000  30.000000  43.153846   
2015-01-11 05:00:00+00:00  30.000000  30.000000  43.153846  43.588694   
2015-01-11 06:00:00+00:00  30.000000  43.153846  43.588694  43.537764   
2015-01-11 08:00:00+00:00  43.153846  43.588694  43.537764  48.252186   
...                              ...        ...        ...        ...   
2018-12-30 12:00:00+00:00  68.057669  69.568080  73.739284  76.254406   
2018-12-30 13:00:00+00:00  69.568080  73.739284  76.254406  67.158844   
2018-12-30 14:00:00+00:00  73.739284  76.254406  67.158844  61.551377   
2018-12-30 15:00:00+00:00  76.254406  67.158844  61.551377  57.997148   
2018-12-30 16:00:00+00:00  67.158844  61.551377  57.997148  62.701536   

Feature                                          ...                      \
Type                                        -69  ...       -8              
Lag                             LOAD        ID3  ...       ID3      LOAD   
Timestamp                                        ...                       
2015-01-11 03:00:00+00:00  43.153846  43.588694  ...  12552.00  12653.50   
2015-01-11 04:00:00+00:00  43.588694  43.537764  ...  12653.50  13797.25   
2015-01-11 05:00:00+00:00  43.537764  48.252186  ...  13797.25  13601.25   
2015-01-11 06:00:00+00:00  48.252186  48.683607  ...  13601.25  13058.25   
2015-01-11 08:00:00+00:00  48.683607  46.580903  ...  13058.25  12296.75   
...                              ...        ...  ...       ...       ...   
2018-12-30 12:00:00+00:00  67.158844  61.551377  ...  10629.00  10633.25   
2018-12-30 13:00:00+00:00  61.551377  57.997148  ...  10633.25  10287.00   
2018-12-30 14:00:00+00:00  57.997148  62.701536  ...  10287.00  10155.75   
2018-12-30 15:00:00+00:00  62.701536  60.599674  ...  10155.75  10121.25   
2018-12-30 16:00:00+00:00  60.599674  60.213662  ...  10121.25  10276.00   

Feature                                                                      \
Type                            -7                  -6                  -5    
Lag                             ID3      LOAD       ID3      LOAD       ID3   
Timestamp                                                                     
2015-01-11 03:00:00+00:00  13797.25  13601.25  13058.25  12296.75  11478.00   
2015-01-11 04:00:00+00:00  13601.25  13058.25  12296.75  11478.00  10813.25   
2015-01-11 05:00:00+00:00  13058.25  12296.75  11478.00  10813.25  10076.75   
2015-01-11 06:00:00+00:00  12296.75  11478.00  10813.25  10076.75   9351.00   
2015-01-11 08:00:00+00:00  11478.00

## Split train and test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    frame['X'],
    frame['y'],
    test_size = 0.3,
    random_state = 0,
    shuffle = False)

X_train.columns = pd.MultiIndex.from_product([['X'], lag_X, X.columns], names = ['Lag', 'Feature', 'Lag'])
y_train.columns = pd.MultiIndex.from_product([['y'], lag_y, y.columns], names = ['Lag', 'Feature', 'Lag'])

frame_train = pd.merge(y_train, X_train, left_index = True, right_index = True)

X_test.columns = pd.MultiIndex.from_product([['X'], lag_X, X.columns], names = ['Lag', 'Feature', 'Lag'])
y_test.columns = pd.MultiIndex.from_product([['y'], lag_y, y.columns], names = ['Lag', 'Feature', 'Lag'])

frame_test = pd.merge(y_test, X_test, left_index = True, right_index = True)

print()
print(f'Train input', frame_train['X'].shape, 'output', frame_train['y'].shape)
print()
print(f'Test input', frame_test['X'].shape, 'output', frame_test['y'].shape)
print()

display(frame_train)


Train input (23639, 138) output (23639, 3)

Test input (10132, 138) output (10132, 3)



Lag                                y                                X  \
Feature                           0          1          2         -72   
Lag                              ID3        ID3        ID3        ID3   
Timestamp                                                               
2015-01-11 03:00:00+00:00  24.922597  26.238903  27.002718  22.953776   
2015-01-11 04:00:00+00:00  26.238903  27.002718  30.000000  23.168355   
2015-01-11 05:00:00+00:00  27.002718  30.000000  35.000000  21.000000   
2015-01-11 06:00:00+00:00  30.000000  35.000000  35.000000  30.000000   
2015-01-11 08:00:00+00:00  35.000000  35.000000  35.000000  30.000000   
...                              ...        ...        ...        ...   
2017-10-29 07:00:00+00:00  15.897972  28.726407  42.253968  47.569562   
2017-10-29 08:00:00+00:00  28.726407  42.253968  39.981959  50.342117   
2017-10-29 09:00:00+00:00  42.253968  39.981959  37.668182  53.282098   
2017-10-29 10:00:00+00:00  39.981959  37.668182  39.994760  51.046393   
2017-10-29 11:00:00+00:00  37.668182  39.994760  41.094024  47.998597   

Lag                                                                    \
Feature                                     -71                   -70   
Lag                             LOAD        ID3       LOAD        ID3   
Timestamp                                                               
2015-01-11 03:00:00+00:00  23.168355  21.000000  30.000000  30.000000   
2015-01-11 04:00:00+00:00  21.000000  30.000000  30.000000  43.153846   
2015-01-11 05:00:00+00:00  30.000000  30.000000  43.153846  43.588694   
2015-01-11 06:00:00+00:00  30.000000  43.153846  43.588694  43.537764   
2015-01-11 08:00:00+00:00  43.153846  43.588694  43.537764  48.252186   
...                              ...        ...        ...        ...   
2017-10-29 07:00:00+00:00  50.342117  53.282098  51.046393  47.998597   
2017-10-29 08:00:00+00:00  53.282098  51.046393  47.998597  45.133267   
2017-10-29 09:00:00+00:00  51.046393  47.998597  45.133267  40.789998   
2017-10-29 10:00:00+00:00  47.998597  45.133267  40.789998  38.877500   
2017-10-29 11:00:00+00:00  45.133267  40.789998  38.877500  40.095831   

Lag                                              ...                      \
Feature                                     -69  ...       -8              
Lag                             LOAD        ID3  ...       ID3      LOAD   
Timestamp                                        ...                       
2015-01-11 03:00:00+00:00  43.153846  43.588694  ...  12552.00  12653.50   
2015-01-11 04:00:00+00:00  43.588694  43.537764  ...  12653.50  13797.25   
2015-01-11 05:00:00+00:00  43.537764  48.252186  ...  13797.25  13601.25   
2015-01-11 06:00:00+00:00  48.252186  48.683607  ...  13601.25  13058.25   
2015-01-11 08:00:00+00:00  48.683607  46.580903  ...  13058.25  12296.75   
...                              ...        ...  ...       ...       ...   
2017-10-29 07:00:00+00:00  45.133267  40.789998  ...  13412.75  12756.50   
2017-10-29 08:00:00+00:00  40.789998  38.877500  ...  12756.50  12091.75   
2017-10-29 09:00:00+00:00  38.877500  40.095831  ...  12091.75  11434.00   
2017-10-29 10:00:00+00:00  40.095831  58.901518  ...  11434.00  10751.00   
2017-10-29 11:00:00+00:00  58.901518  54.114334  ...  10751.00  10082.50   

Lag                                                                          \
Feature                         -7                  -6                  -5    
Lag                             ID3      LOAD       ID3      LOAD       ID3   
Timestamp                                                                     
2015-01-11 03:00:00+00:00  13797.25  13601.25  13058.25  12296.75  11478.00   
2015-01-11 04:00:00+00:00  13601.25  13058.25  12296.75  11478.00  10813.25   
2015-01-11 05:00:00+00:00  13058.25  12296.75  11478.00  10813.25  10076.75   
2015-01-11 06:00:00+00:00  12296.75  11478.00  10813.25  10076.75   9351.00   
2015-01-11 08:00:00+00:00  11478.00

## Scaling

In [6]:
frame_train_unscaled = frame_train
frame_test_unscaled = frame_test

y_scaler = StandardScaler()
y_scaler.fit(frame_train['y'])

frame_train['y'] = y_scaler.transform(frame_train['y'])
frame_test['y'] = y_scaler.transform(frame_test['y'])

X_scaler = StandardScaler()
X_scaler.fit(frame_train['X'])

frame_train['X'] = X_scaler.transform(frame_train['X'])
frame_test['X'] = X_scaler.transform(frame_test['X'])

display(frame_train)

display(frame_test)

Lag                               y                             X            \
Feature                          0         1         2        -72             
Lag                             ID3       ID3       ID3       ID3      LOAD   
Timestamp                                                                     
2015-01-11 03:00:00+00:00 -0.883980 -0.798055 -0.748213 -1.015689 -1.001737   
2015-01-11 04:00:00+00:00 -0.798000 -0.748162 -0.552428 -1.001658 -1.143530   
2015-01-11 05:00:00+00:00 -0.748109 -0.552380 -0.225824 -1.143448 -0.555003   
2015-01-11 06:00:00+00:00 -0.552329 -0.225780 -0.225824 -0.554931 -0.555003   
2015-01-11 08:00:00+00:00 -0.225734 -0.225780 -0.225824 -0.554931  0.305152   
...                             ...       ...       ...       ...       ...   
2017-10-29 07:00:00+00:00 -1.473459 -0.635571  0.248012  0.593955  0.775207   
2017-10-29 08:00:00+00:00 -0.635519  0.248050  0.099602  0.775254  0.967458   
2017-10-29 09:00:00+00:00  0.248088  0.099642 -0.051536  0.967502  0.821261   
2017-10-29 10:00:00+00:00  0.099682 -0.051494  0.100439  0.821307  0.621960   
2017-10-29 11:00:00+00:00 -0.051451  0.100478  0.172243  0.622010  0.434590   

Lag                                                                          \
Feature                         -71                 -70                 -69   
Lag                             ID3      LOAD       ID3      LOAD       ID3   
Timestamp                                                                     
2015-01-11 03:00:00+00:00 -1.143602 -0.555128 -0.555159  0.304953  0.333353   
2015-01-11 04:00:00+00:00 -0.555063 -0.555128  0.305043  0.333390  0.330022   
2015-01-11 05:00:00+00:00 -0.555063  0.305069  0.333480  0.330059  0.638308   
2015-01-11 06:00:00+00:00  0.305110  0.333506  0.330149  0.638351  0.666520   
2015-01-11 08:00:00+00:00  0.333546  0.330175  0.638451  0.666563  0.529019   
...                             ...       ...       ...       ...       ...   
2017-10-29 07:00:00+00:00  0.967429  0.821202  0.621867  0.434394  0.150340   
2017-10-29 08:00:00+00:00  0.821229  0.621892  0.434488  0.150373  0.025278   
2017-10-29 09:00:00+00:00  0.621924  0.434513  0.150458  0.025308  0.104947   
2017-10-29 10:00:00+00:00  0.434551  0.150485  0.025389  0.104979  1.334690   
2017-10-29 11:00:00+00:00  0.150530  0.025417  0.105062  1.334748  1.021646   

Lag                        ...                                          \
Feature                    ...       -8                  -7              
Lag                        ...       ID3      LOAD       ID3      LOAD   
Timestamp                  ...                                           
2015-01-11 03:00:00+00:00  ...  0.097109  0.138955  0.610075  0.529413   
2015-01-11 04:00:00+00:00  ...  0.138915  0.610038  0.529349  0.305774   
2015-01-11 05:00:00+00:00  ...  0.610006  0.529311  0.305706 -0.007856   
2015-01-11 06:00:00+00:00  ...  0.529277  0.305662 -0.007931 -0.345065   
2015-01-11 08:00:00+00:00  ...  0.305625 -0.007981 -0.345146 -0.618848   
...                        ...       ...       ...       ...       ...   
2017-10-29 07:00:00+00:00  ...  0.451637  0.181379 -0.092363 -0.363187   
2017-10-29 08:00:00+00:00  ...  0.181339 -0.092416 -0.363268 -0.644486   
2017-10-29 09:00:00+00:00  ... -0.092460 -0.363327 -0.644573 -0.919813   
2017-10-29 10:00:00+00:00  ... -0.363376 -0.644638 -0.919905 -1.055005   
2017-10-29 11:00:00+00:00  ... -0.644693 -0.919977 -1.055100 -1.143143   

Lag                                                                          \
Feature                         -6                  -5                  -4    
Lag                             ID3      LOAD       ID3      LOAD       ID3   
Timestamp                                                                     
2015-01-11 03:00:00+00:00  0.305839 -0.007720 -0.344854 -0.618579 -0.921875   
2015-01-11 04:00:00+00:00 -0.007783 -0.344911 -0.618616 -0.921884 -1.220751   
2015-01-11 05:00:00+00:00 -0.344983 -0.6186

Lag                               y                             X            \
Feature                          0         1         2        -72             
Lag                             ID3       ID3       ID3       ID3      LOAD   
Timestamp                                                                     
2017-10-29 12:00:00+00:00  0.100518  0.172282  0.432890  0.434644  0.150576   
2017-10-29 13:00:00+00:00  0.172321  0.432925  0.781620  0.150634  0.025514   
2017-10-29 14:00:00+00:00  0.432960  0.781651  1.650408  0.025575  0.105183   
2017-10-29 15:00:00+00:00  0.781680  1.650428  1.734947  0.105242  1.334922   
2017-10-29 16:00:00+00:00  1.650443  1.734966  1.153214  1.334960  1.021879   
...                             ...       ...       ...       ...       ...   
2018-12-30 12:00:00+00:00  1.157089  0.863447  1.001604  1.867580  1.933660   
2018-12-30 13:00:00+00:00  0.863475  1.001632  1.373078  1.933687  2.032429   
2018-12-30 14:00:00+00:00  1.001658  1.373101  1.399610  2.032454  2.305192   
2018-12-30 15:00:00+00:00  1.373120  1.399633  1.372677  2.305212  2.469660   
2018-12-30 16:00:00+00:00  1.399652  1.372700  1.305883  2.469678  1.874884   

Lag                                                                          \
Feature                         -71                 -70                 -69   
Lag                             ID3      LOAD       ID3      LOAD       ID3   
Timestamp                                                                     
2017-10-29 12:00:00+00:00  0.025466  0.105090  1.334869  1.021697  0.842035   
2017-10-29 13:00:00+00:00  0.105136  1.334889  1.021809  0.842082  0.255591   
2017-10-29 14:00:00+00:00  1.334901  1.021831  0.842188  0.255626 -0.341605   
2017-10-29 15:00:00+00:00  1.021852  0.842211  0.255714 -0.341581 -0.631051   
2017-10-29 16:00:00+00:00  0.842237  0.255740 -0.341512 -0.631034 -0.633566   
...                             ...       ...       ...       ...       ...   
2018-12-30 12:00:00+00:00  2.032422  2.305205  2.469669  1.874722  1.507970   
2018-12-30 13:00:00+00:00  2.305191  2.469682  1.874861  1.508031  1.275552   
2018-12-30 14:00:00+00:00  2.469663  1.874877  1.508158  1.275608  1.583182   
2018-12-30 15:00:00+00:00  1.874875  1.508176  1.275727  1.583244  1.445736   
2018-12-30 16:00:00+00:00  1.508184  1.275747  1.583373  1.445796  1.420494   

Lag                        ...                                          \
Feature                    ...       -8                  -7              
Lag                        ...       ID3      LOAD       ID3      LOAD   
Timestamp                  ...                                           
2017-10-29 12:00:00+00:00  ... -0.920036 -1.055175 -1.143240 -1.200288   
2017-10-29 13:00:00+00:00  ... -1.055237 -1.143317 -1.200386 -1.185358   
2017-10-29 14:00:00+00:00  ... -1.143380 -1.200464 -1.185456 -1.120491   
2017-10-29 15:00:00+00:00  ... -1.200529 -1.185534 -1.120587 -0.957910   
2017-10-29 16:00:00+00:00  ... -1.185598 -1.120664 -0.958003 -0.743023   
...                        ...       ...       ...       ...       ...   
2018-12-30 12:00:00+00:00  ... -0.694942 -0.693137 -0.835679 -0.889644   
2018-12-30 13:00:00+00:00  ... -0.693192 -0.835748 -0.889736 -0.903853   
2018-12-30 14:00:00+00:00  ... -0.835806 -0.889807 -0.903945 -0.840118   
2018-12-30 15:00:00+00:00  ... -0.889866 -0.904017 -0.840209 -0.711413   
2018-12-30 16:00:00+00:00  ... -0.904076 -0.840279 -0.711501 -0.515574   

Lag                                                                          \
Feature                         -6                  -5                  -4    
Lag                             ID3      LOAD       ID3      LOAD       ID3   
Timestamp                                                                     
2017-10-29 12:00:00+00:00 -1.185256 -1.120296 -0.957651 -0.742743 -0.463831   
2017-10-29 13:00:00+00:00 -1.120390 -0.957723 -0.742781 -0.463838 -0.247421   
2017-10-29 14:00:00+00:00 -0.957813 -0.7428

In [7]:
X_train = list()

for index, row in frame_train['X'].iterrows():
    X_train.extend(row.tolist())

X_train = np.array(X_train)

X_train = X_train.reshape((len(frame_train), len(lag_X), len(X.columns)))

X_test = list()

for index, row in frame_test['X'].iterrows():
    X_test.extend(row.tolist())

X_test = np.array(X_test)

X_test = X_test.reshape((len(frame_test), len(lag_X), len(X.columns)))

In [8]:
# step1 = []
# step2 = []
# step3 = []

# for index, row in frame_train.iterrows():
#     step2 = []
#     for l in lag_X:
#         step1 = []
#         for c in X.columns:
#             step1.append(row['X'][f'{c}'][l])
#         step2.append(step1)
#     step3.append(step2)

# X_train = step3

# X_train = np.array(X_train)

In [9]:
# step1 = []
# step2 = []
# step3 = []

# for index, row in frame_test.iterrows():
#     step2 = []
#     for l in lag_X:
#         step1 = []
#         for c in X.columns:
#             step1.append(row['X'][f'{c}'][l])
#         step2.append(step1)
#     step3.append(step2)

# X_test = step3

# X_test = np.array(X_test)

# Learning

## Create model

In [10]:
LATENT_DIM = 5
BATCH_SIZE = 32
EPOCHS = 10

In [11]:
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.optimizers import SGD, Adam
from keras.utils.vis_utils import plot_model
from keras.layers import GRU, Dense, RepeatVector, TimeDistributed, Flatten

In [12]:
T = len(lag_X)
HORIZON = len(lag_y)

model = Sequential()

model.add(GRU(LATENT_DIM, input_shape = (T, len(X.columns))))

model.add(RepeatVector(HORIZON))

model.add(GRU(LATENT_DIM, return_sequences = True))

model.add(TimeDistributed(Dense(1)))

model.add(Flatten())

In [13]:
model.compile(optimizer = 'Adam', loss = 'mse')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 5)                 135       
_________________________________________________________________
repeat_vector (RepeatVector) (None, 3, 5)              0         
_________________________________________________________________
gru_1 (GRU)                  (None, 3, 5)              180       
_________________________________________________________________
time_distributed (TimeDistri (None, 3, 1)              6         
_________________________________________________________________
flatten (Flatten)            (None, 3)                 0         
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_train,
          np.array(frame_train['y']),
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
        #   validation_data=(valid_inputs['X'], valid_inputs['target']),
        #   callbacks=[earlystop],
          verbose=1)

Epoch 1/10
739/739 [==============================] - 25s 27ms/step - loss: 0.8889
Epoch 2/10
739/739 [==============================] - 21s 29ms/step - loss: 0.8048
Epoch 3/10
739/739 [==============================] - 20s 27ms/step - loss: 0.8568
Epoch 4/10
739/739 [==============================] - 19s 25ms/step - loss: 0.7565
Epoch 5/10
739/739 [==============================] - 17s 22ms/step - loss: 0.8460
Epoch 6/10
739/739 [==============================] - 16s 22ms/step - loss: 0.8092
Epoch 7/10
739/739 [==============================] - 16s 22ms/step - loss: 0.8156
Epoch 8/10
739/739 [==============================] - 17s 23ms/step - loss: 0.7764
Epoch 9/10
739/739 [==============================] - 17s 22ms/step - loss: 0.7783
Epoch 10/10
739/739 [==============================] - 16s 22ms/step - loss: 0.7717


In [15]:
predictions = pd.DataFrame(index = frame_test.index, columns = pd.MultiIndex.from_product([['Prediction'], y.columns, lag_y], names = ['Type', 'Feature', 'Lag']))

frame_test = pd.merge(frame_test, predictions, left_index = True, right_index = True)

frame_test['Prediction'] = model.predict(X_test)

frame_test['Prediction'] = y_scaler.inverse_transform(frame_test['Prediction'])

frame_test['y'] = y_scaler.inverse_transform(frame_test['y'])

display(frame_test)

Lag                                y                               X  \
Feature                            0          1          2       -72   
Lag                              ID3        ID3        ID3       ID3   
Timestamp                                                              
2017-10-29 12:00:00+00:00  39.994760  41.094024  45.084266  0.434644   
2017-10-29 13:00:00+00:00  41.094024  45.084266  50.422995  0.150634   
2017-10-29 14:00:00+00:00  45.084266  50.422995  63.723309  0.025575   
2017-10-29 15:00:00+00:00  50.422995  63.723309  65.017521  0.105242   
2017-10-29 16:00:00+00:00  63.723309  65.017521  56.111738  1.334960   
...                              ...        ...        ...       ...   
2018-12-30 12:00:00+00:00  56.170316  51.675229  53.790740  1.867580   
2018-12-30 13:00:00+00:00  51.675229  53.790740  59.477646  1.933687   
2018-12-30 14:00:00+00:00  53.790740  59.477646  59.883829  2.032454   
2018-12-30 15:00:00+00:00  59.477646  59.883829  59.471501  2.305212   
2018-12-30 16:00:00+00:00  59.883829  59.471501  58.448950  2.469678   

Lag                                                                          \
Feature                                   -71                 -70             
Lag                            LOAD       ID3      LOAD       ID3      LOAD   
Timestamp                                                                     
2017-10-29 12:00:00+00:00  0.150576  0.025466  0.105090  1.334869  1.021697   
2017-10-29 13:00:00+00:00  0.025514  0.105136  1.334889  1.021809  0.842082   
2017-10-29 14:00:00+00:00  0.105183  1.334901  1.021831  0.842188  0.255626   
2017-10-29 15:00:00+00:00  1.334922  1.021852  0.842211  0.255714 -0.341581   
2017-10-29 16:00:00+00:00  1.021879  0.842237  0.255740 -0.341512 -0.631034   
...                             ...       ...       ...       ...       ...   
2018-12-30 12:00:00+00:00  1.933660  2.032422  2.305205  2.469669  1.874722   
2018-12-30 13:00:00+00:00  2.032429  2.305191  2.469682  1.874861  1.508031   
2018-12-30 14:00:00+00:00  2.305192  2.469663  1.874877  1.508158  1.275608   
2018-12-30 15:00:00+00:00  2.469660  1.874875  1.508176  1.275727  1.583244   
2018-12-30 16:00:00+00:00  1.874884  1.508184  1.275747  1.583373  1.445796   

Lag                                  ...                                \
Feature                         -69  ...        -7        -6             
Lag                             ID3  ...      LOAD       ID3      LOAD   
Timestamp                            ...                                 
2017-10-29 12:00:00+00:00  0.842035  ... -1.200288 -1.185256 -1.120296   
2017-10-29 13:00:00+00:00  0.255591  ... -1.185358 -1.120390 -0.957723   
2017-10-29 14:00:00+00:00 -0.341605  ... -1.120491 -0.957813 -0.742848   
2017-10-29 15:00:00+00:00 -0.631051  ... -0.957910 -0.742931 -0.463931   
2017-10-29 16:00:00+00:00 -0.633566  ... -0.743023 -0.464007 -0.247511   
...                             ...  ...       ...       ...       ...   
2018-12-30 12:00:00+00:00  1.507970  ... -0.889644 -0.903758 -0.839938   
2018-12-30 13:00:00+00:00  1.275552  ... -0.903853 -0.840025 -0.711239   
2018-12-30 14:00:00+00:00  1.583182  ... -0.840118 -0.711322 -0.515411   
2018-12-30 15:00:00+00:00  1.445736  ... -0.711413 -0.515488 -0.217756   
2018-12-30 16:00:00+00:00  1.420494  ... -0.515574 -0.217825  0.079075   

Lag                                                               Prediction  \
Feature                          -5                  -4                  ID3   
Lag                             ID3      LOAD       ID3      LOAD          0   
Timestamp                                                                      
2017-10-29 12:00:00+00:00 -0.957651 -0.742743 -0.463831 -0.247444  32.864662   
2017-10-29 13:00:00+00:00 -0.742781 -0.463838 -0.247421 -0.171257  30.855253   
2017-10-29 14:00:00+00:00 -0.463872 -0.247428 -0.171235 -0.144077  30.943216   
2017-10-29 15:00:00+00:00 -0.247458 -0.171241 -0.144055 -0.123

# Results

## Plot prediction

In [16]:
import plotly.express as px
import plotly.graph_objects as go

fig1 = go.Scatter(      x = frame_test.index,
                        y = frame_test['y'][0]['ID3'],
                        name = 'Actual',
                        # color = hex_maroon
                        # title = "Log of Appliance Energy Consumption in Wh vs Time"
                    )

fig2 = go.Scatter(      x = frame_test.index,
                        y = frame_test['Prediction']['ID3'][0],
                        name = 'Predicted',
                        # color = hex_gold
                        # title = "Log of Appliance Energy Consumption in Wh vs Time"
                    )

data = [fig1, fig2]

fig = go.Figure(data = data)

fig.update_layout(      title = 'Forecast of test set',
                        xaxis_title = 'Timestamp',
                        yaxis_title = 'ID3 (€)')

fig.show()

## Metrics

In [17]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

print(smape(frame_test['y'][0]['ID3'], frame_test['Prediction']['ID3'][0]))
print(smape(frame_test['y'][1]['ID3'], frame_test['Prediction']['ID3'][1]))
print(smape(frame_test['y'][2]['ID3'], frame_test['Prediction']['ID3'][2]))

39.97994890163044
41.39233838824775
41.41276997047666


In [18]:
from sklearn.metrics import mean_absolute_error

display(mean_absolute_error(frame_test['y'][0]['ID3'], frame_test['Prediction']['ID3'][0]))

18.7331313607741